## Final Project - Python Data Science

In [192]:
# Import library pandas dan file raw data yang dibutuhkan
import pandas as pd
import numpy as np

df_train = pd.read_csv('https://raw.githubusercontent.com/multimediary/dataset/master/train.csv')
df_test = pd.read_csv('https://raw.githubusercontent.com/multimediary/dataset/master/test.csv')

In [193]:
# Melihat ukuran dimensi dataframe
print(df_train.shape) # (Baris, Kolom)
print(df_test.shape) # (Baris, Kolom)

(35994, 13)
(9599, 12)


In [194]:
# Membuat fungsi concat Dataframe
def concat_df(train_data, test_data):
    # Returns a concatenated df of training and test set
    return pd.concat([train_data, test_data], sort=True).reset_index(drop=True)

df_all = concat_df(df_train, df_test)

In [195]:
df_all.head()

,Berat Akhir,Gaji,Jam per Minggu,Jenis Kelamin,Jmlh Tahun Pendidikan,Kelas Pekerja,Kerugian Capital,Keuntungan Kapital,Pekerjaan,Pendidikan,Status Perkawinan,Umur,id
0,242912,<=7jt,35.0,Perempuan,9,Wiraswasta,0.0,0.0,Servis Lainnya,SMA,Belum Pernah Menikah,21,0
1,140782,<=7jt,40.0,Perempuan,6,Wiraswasta,0.0,0.0,Ekesekutif Managerial,10th,Cerai,49,1
2,120057,>7jt,45.0,Laki2,12,Wiraswasta,0.0,61404000.0,Ekesekutif Managerial,D3,Menikah,44,2
3,194630,<=7jt,35.0,Laki2,13,Wiraswasta,0.0,0.0,Spesialis,Sarjana,Belum Pernah Menikah,24,3
4,219619,>7jt,40.0,Laki2,14,Wiraswasta,0.0,210336000.0,Spesialis,Master,Menikah,33,4


In [196]:
#Data test
df_all.loc[35994:]

,Berat Akhir,Gaji,Jam per Minggu,Jenis Kelamin,Jmlh Tahun Pendidikan,Kelas Pekerja,Kerugian Capital,Keuntungan Kapital,Pekerjaan,Pendidikan,Status Perkawinan,Umur,id
35994,99761,NaN,40.0,Perempuan,11,Pemerintah Lokal,0.0,0.0,Pemuka Agama,D4,Cerai,30,35994
35995,202206,NaN,50.0,Laki2,9,Wiraswasta,0.0,0.0,Mesin Inspeksi,SMA,Menikah,28,35995
35996,199934,NaN,40.0,Laki2,10,Pemerintah Provinsi,0.0,0.0,Pemuka Agama,Pendidikan Tinggi,Menikah,34,35996
35997,373448,NaN,40.0,Laki2,10,Wiraswasta,28028000.0,0.0,Ekesekutif Managerial,Pendidikan Tinggi,Menikah,51,35997
35998,144685,NaN,40.0,Perempuan,10,?,22428000.0,0.0,?,Pendidikan Tinggi,Belum Pernah Menikah,20,35998
...,...,...,...,...,...,...,...,...,...,...,...,...,...
45588,159021,NaN,40.0,Laki2,5,Pemerintah Negara,0.0,0.0,Petani,9th,Menikah,58,45588
45589,144086,NaN,50.0,Laki2,7,Pekerja Bebas Bukan Perusahan,0.0,0.0,Perbaikan Kerajinan,11th,Menikah,45,45589
45590,238685,NaN,32.0,Perempuan,10,Wiraswasta,0.0,0.0,Servis Lainnya,Pendidikan Tinggi,Belum Pernah Menikah,20,45590
45591,249039,NaN,40.0,Perempuan,9,Pemerintah Lokal,0.0,0.0,Pemuka Agama,SMA,Berpisah,41,45591


## 1. Feature Discovering
Mengenali missing value dari setiap kolom

In [197]:
# Buat fungsi untuk menampilkan jumlah missing value
def display_missing(df):    
    for col in df.columns.tolist():          
        print('{} column missing values: {}'.format(col, df[col].isnull().sum()))
    print('\n')

In [198]:
# Cek missing value data train
display_missing(df_all)

Berat Akhir column missing values: 0
Gaji column missing values: 9599
Jam per Minggu column missing values: 0
Jenis Kelamin column missing values: 0
Jmlh Tahun Pendidikan column missing values: 0
Kelas Pekerja column missing values: 0
Kerugian Capital column missing values: 0
Keuntungan Kapital column missing values: 0
Pekerjaan column missing values: 0
Pendidikan column missing values: 0
Status Perkawinan column missing values: 0
Umur column missing values: 0
id column missing values: 0




### 1.1 Kolom umur

In [199]:
# Cek kolom umur
df_all['Umur'].describe()

count    45593.000000
mean        38.618801
std         13.698814
min         17.000000
25%         28.000000
50%         37.000000
75%         48.000000
max         90.000000
Name: Umur, dtype: float64

In [200]:
bins= [0,17,25,35,50,100]
labels = ['0','1','2','3','4']
df_all['AgeGroup'] = pd.cut(df_all['Umur'], bins=bins, labels=labels, right=False)

In [201]:
df_all['AgeGroup'].value_counts()

3    16048
2    11753
4     9918
1     7874
0        0
Name: AgeGroup, dtype: int64

In [202]:
df_all.drop('Umur', axis=1, inplace=True)

### 1.2 Kolom Kelas Pekerja

In [203]:
df_all['Kelas Pekerja'].value_counts()

Wiraswasta                       31662
Pekerja Bebas Bukan Perusahan     3598
Pemerintah Lokal                  2904
?                                 2615
Pemerintah Negara                 1853
Pekerja Bebas Perusahaan          1589
Pemerintah Provinsi               1345
Tanpa di Bayar                      19
Tidak Pernah Bekerja                 8
Name: Kelas Pekerja, dtype: int64

In [204]:
# Set kolom ? dengan nilai modus
df_all['Kelas Pekerja'].loc[df_all['Kelas Pekerja']=='?'] = 'Wiraswasta'

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [205]:
df_all['Kelas Pekerja'].value_counts()

Wiraswasta                       34277
Pekerja Bebas Bukan Perusahan     3598
Pemerintah Lokal                  2904
Pemerintah Negara                 1853
Pekerja Bebas Perusahaan          1589
Pemerintah Provinsi               1345
Tanpa di Bayar                      19
Tidak Pernah Bekerja                 8
Name: Kelas Pekerja, dtype: int64

### 1.3 Kolom Berat Akhir

In [206]:
df_all['Berat Akhir'].describe()

count    4.559300e+04
mean     1.899454e+05
std      1.059312e+05
min      1.228500e+04
25%      1.175840e+05
50%      1.783190e+05
75%      2.379430e+05
max      1.490400e+06
Name: Berat Akhir, dtype: float64

In [207]:
berat_label= [1,2,3,4,5,6,7,8,9,10]
df_all['Berat Akhir']=pd.qcut(df_all['Berat Akhir'],10, labels=berat_label)

In [208]:
# Kolom ini blm tau kelanjutan prosesnya

### 1.4 Kolom Pendidikan

In [209]:
df_all['Pendidikan'].value_counts()

SMA                     14756
Pendidikan Tinggi       10181
Sarjana                  7492
Master                   2453
D4                       1917
11th                     1694
D3                       1487
10th                     1295
7th-8th                   896
Sekolah Professional      774
9th                       708
12th                      603
Doktor                    555
5th-6th                   474
1st-4th                   231
SD                         77
Name: Pendidikan, dtype: int64

In [210]:
# Set beberapa data ambigu ke Non SMA
dict_pend = {'11th':'Non SMA','10th':'Non SMA','7th-8th':'Non SMA','9th':'Non SMA',
             '12th':'Non SMA','5th-6th':'Non SMA','1st-4th':'Non SMA','SD':'Non SMA'}
df_all['Pendidikan'] = df_all['Pendidikan'].map(dict_pend).fillna(df_all['Pendidikan']) 

In [211]:
df_all['Pendidikan'].value_counts()

SMA                     14756
Pendidikan Tinggi       10181
Sarjana                  7492
Non SMA                  5978
Master                   2453
D4                       1917
D3                       1487
Sekolah Professional      774
Doktor                    555
Name: Pendidikan, dtype: int64

### 1.5 Kolom Jmlh Tahun Pendidikan

In [212]:
df_all['Jmlh Tahun Pendidikan'].value_counts()

9     14756
10    10181
13     7492
14     2453
11     1917
7      1694
12     1487
6      1295
4       896
15      774
5       708
8       603
16      555
3       474
2       231
1        77
Name: Jmlh Tahun Pendidikan, dtype: int64

In [213]:
# Isi data hampir sama dengan kolom pendidikan
# Sebaikanya tidak perlu dimasukan ke dalam features

In [214]:
df_all.drop('Jmlh Tahun Pendidikan', axis=1, inplace=True)

### 1.6 Kolom Status Perkawinan

In [215]:
df_all['Status Perkawinan'].value_counts()

Menikah                 20892
Belum Pernah Menikah    15064
Cerai                    6161
Berpisah                 1434
Janda                    1416
Menikah LDR               626
Name: Status Perkawinan, dtype: int64

In [216]:
# Tidak perlu di lakukan feature engineering

### 1.7 Kolom Pekerjaan

In [217]:
df_all['Pekerjaan'].value_counts()

Spesialis                5729
Perbaikan Kerajinan      5712
Ekesekutif Managerial    5669
Pemuka Agama             5270
Sales                    5123
Servis Lainnya           4597
Mesin Inspeksi           2840
?                        2623
Supir                    2199
Pembersih                1938
Petani                   1392
Tech-support             1343
Penjaga                   914
Asisten Rumah Tangga      230
Tentara                    14
Name: Pekerjaan, dtype: int64

### 1.8 Kolom Jenis Kelamin

In [218]:
df_all['Jenis Kelamin'].value_counts()

Laki2        30474
Perempuan    15119
Name: Jenis Kelamin, dtype: int64

### 1.9 Kolom Keuntungan Kapital

In [219]:
df_all['Keuntungan Kapital'].describe()

count    4.559300e+04
mean     1.500785e+07
std      1.032283e+08
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.399986e+09
Name: Keuntungan Kapital, dtype: float64

In [220]:
a, b  = pd.cut(df_all['Keuntungan Kapital'],10 ,labels=[1,2,3,4,5,6,7,8,9,10], retbins=True)
df_all['Keuntungan Kapital']= pd.cut(df_all['Keuntungan Kapital'], bins=b,labels=[1,2,3,4,5,6,7,8,9,10])

### 1.10 Kolom Kerugian Capital

In [221]:
df_all['Kerugian Capital'].describe()

count    4.559300e+04
mean     1.225298e+06
std      5.648968e+06
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      6.098400e+07
Name: Kerugian Capital, dtype: float64

In [222]:
a, b  = pd.cut(df_all['Kerugian Capital'],10 ,labels=[1,2,3,4,5,6,7,8,9,10], retbins=True)
df_all['Kerugian Capital']= pd.cut(df_all['Kerugian Capital'], bins=b,labels=[1,2,3,4,5,6,7,8,9,10])

### 1.11 Kolom Jam per Minggu

In [223]:
df_all['Jam per Minggu'].describe()

count    45593.000000
mean        40.413594
std         12.374100
min          1.000000
25%         40.000000
50%         40.000000
75%         45.000000
max         99.000000
Name: Jam per Minggu, dtype: float64

In [224]:
a, b  = pd.cut(df_all['Jam per Minggu'],3 ,labels=[1,2,3], retbins=True)
df_all['Jam per Minggu']= pd.cut(df_all['Jam per Minggu'], bins=b,labels=[1,2,3])

## 2. Modeling

### 2.1 Scalling values

In [225]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [226]:
# non_numeric_features = ['AgeGroup','Kelas Pekerja','Pendidikan','Status Perkawinan','Pekerjaan','Jenis Kelamin']
non_numeric_features = ['Kelas Pekerja','Pendidikan','Status Perkawinan','Pekerjaan','Jenis Kelamin']

In [227]:
for feature in non_numeric_features:        
    df_all[feature] = LabelEncoder().fit_transform(df_all[feature])

In [228]:
df_all.head()

,Berat Akhir,Gaji,Jam per Minggu,Jenis Kelamin,Kelas Pekerja,Kerugian Capital,Keuntungan Kapital,Pekerjaan,Pendidikan,Status Perkawinan,id,AgeGroup
0,8,<=7jt,2,1,7,1,1,10,6,0,0,1
1,4,<=7jt,2,1,7,1,1,2,4,2,1,3
2,3,>7jt,2,0,7,1,1,2,0,4,2,3
3,6,<=7jt,2,0,7,1,1,11,7,0,3,1
4,7,>7jt,2,0,7,1,2,11,3,4,4,2


In [229]:
def normalize(x):
    a = x - x.min()
    b = x.max() - x.min()
    return a/b

In [230]:
# df_all['Berat Akhir'] = df_all[['Berat Akhir']].apply(normalize)
# df_all['Kerugian Capital'] = df_all[['Kerugian Capital']].apply(normalize)
# df_all['Keuntungan Kapital'] = df_all[['Keuntungan Kapital']].apply(normalize)
# df_all['Jam per Minggu'] = df_all[['Jam per Minggu']].apply(normalize)

In [231]:
df_train = df_all.loc[:35993]
df_test = df_all.loc[35994:]

In [232]:
print(df_train.shape)
print(df_test.shape)

(35994, 12)
(9599, 12)


In [233]:
X = StandardScaler().fit_transform(df_train.drop(columns=['Gaji']))

In [234]:
X.shape

(35994, 11)

In [235]:
dict_gaji = {'<=7jt':'0','>7jt':'1'}
df_train['Gaji'] = df_train['Gaji'].map(dict_gaji).fillna(df_train['Gaji']) 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [236]:
df_train['Gaji'].value_counts()

0    27364
1     8630
Name: Gaji, dtype: int64

In [237]:
y = df_train['Gaji'].values

In [238]:
y

array(['0', '0', '1', ..., '1', '0', '1'], dtype=object)

### 2.2 Algoritma

In [239]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [240]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=21, stratify=y)

### A. NaiveBayes

In [241]:
from sklearn.naive_bayes import GaussianNB
model_nb = GaussianNB()

cv_score = cross_val_score(model_nb, X, y, cv=10, scoring='roc_auc')
cv_score.mean()

0.8332263788108648

In [242]:
model.fit(X_train, y_train)
y_predict_train = model.predict(X_train)
y_predict_test = model.predict(X_test)

In [243]:
GaussianNB?

### B. RandomForest

In [244]:
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(criterion='gini', 
                                           n_estimators=1100,
                                           max_depth=5,
                                           min_samples_split=4,
                                           min_samples_leaf=5,
                                           max_features='auto',
                                           oob_score=True,
                                           random_state=50)
random_forest.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=4,
                       min_weight_fraction_leaf=0.0, n_estimators=1100,
                       n_jobs=None, oob_score=True, random_state=50, verbose=0,
                       warm_start=False)

In [245]:
y_predict_train = random_forest.predict(X_train)
y_predict_test = random_forest.predict(X_test)

In [246]:
from sklearn.metrics import accuracy_score 
print('Score data train', accuracy_score(y_train, y_predict_train))
print('Score data test', accuracy_score(y_test, y_predict_test))

Score data train 0.8307093906278941
Score data test 0.8249805533948217


### C. DecissionTree

In [247]:
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier()

In [248]:
params = {'random_state':np.arange(0,40),'criterion':['gini','entropy'],
          'splitter':['best','random'],'max_depth':np.arange(2,20)} 
rscv = RandomizedSearchCV(dtree, params, scoring='accuracy', cv=10, n_iter=100)
rscv.fit(X_train, y_train)

RandomizedSearchCV(cv=10, error_score=nan,
                   estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort='deprecated',
                                                    random_state=None,
         

In [249]:
print(rscv.best_params_)
print(rscv.best_score_)

{'splitter': 'best', 'random_state': 32, 'max_depth': 8, 'criterion': 'gini'}
0.8377842685219491


In [250]:
# Prediksi data train dan test
y_predict_train = rscv.predict(X_train)
y_predict_test = rscv.predict(X_test)

In [251]:
from sklearn.metrics import accuracy_score 
print('Score data train', accuracy_score(y_train, y_predict_train))
print('Score data test', accuracy_score(y_test, y_predict_test))

Score data train 0.8457121689201704
Score data test 0.8362040226691855


### 2.3 Tuning hyperparameter

In [252]:
from sklearn.neighbors import KNeighborsClassifier
model_knn = KNeighborsClassifier(n_neighbors=9, weights='distance')
params = {'n_neighbors':np.arange(5,10),'weights':['uniform','distance']}
gscv = GridSearchCV(model_knn,param_grid=params,scoring='roc_auc',cv=10)
gscv.fit(X, y)

GridSearchCV(cv=10, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=9, p=2,
                                            weights='distance'),
             iid='deprecated', n_jobs=None,
             param_grid={'n_neighbors': array([5, 6, 7, 8, 9]),
                         'weights': ['uniform', 'distance']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='roc_auc', verbose=0)

In [253]:
print(gscv.best_params_)
print(gscv.best_score_)

{'n_neighbors': 9, 'weights': 'distance'}
0.8425037332312917


In [254]:

cv_score = cross_val_score(model_knn, X, y, cv=10, scoring='roc_auc')
cv_score.mean()

0.8425037332312917

In [255]:
model_knn.fit(X_train, y_train)
y_predict_train = model_knn.predict(X_train)
y_predict_test = model_knn.predict(X_test)

In [256]:
from sklearn.metrics import accuracy_score 
print('Score data train', accuracy_score(y_train, y_predict_train))
print('Score data test', accuracy_score(y_test, y_predict_test))

Score data train 1.0
Score data test 0.8117568618735415


## 3. Real Data

In [257]:
df_test.drop('Gaji', axis=1, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [258]:
df_test.head()

,Berat Akhir,Jam per Minggu,Jenis Kelamin,Kelas Pekerja,Kerugian Capital,Keuntungan Kapital,Pekerjaan,Pendidikan,Status Perkawinan,id,AgeGroup
35994,2,2,1,2,1,1,5,1,2,35994,2
35995,7,2,0,7,1,1,3,6,4,35995,2
35996,7,2,0,4,1,1,5,5,4,35996,2
35997,10,2,0,7,5,1,2,5,4,35997,4
35998,4,2,1,7,4,1,0,5,0,35998,1


In [259]:
df_test.shape

(9599, 11)

In [260]:
X_test = StandardScaler().fit_transform(df_test)

In [261]:
df_test_predict = random_forest.predict(X_test)

In [262]:
df_test_predict

array(['0', '0', '0', ..., '0', '0', '0'], dtype=object)

In [263]:
df_predict_series = pd.Series(df_test_predict, index=df_test['id'])

In [264]:
result = df_test.join(df_predict_series.to_frame())

In [265]:
result.set_axis([*result.columns[:-1], 'Gaji'], axis=1, inplace=True)

In [266]:
result['Gaji'].value_counts()

0    8562
1    1037
Name: Gaji, dtype: int64

In [267]:
result_csv = result[['id','Gaji']]
result_csv.head()

,id,Gaji
35994,35994,0
35995,35995,0
35996,35996,0
35997,35997,1
35998,35998,0


In [268]:
result_csv.to_csv(r'Submission.csv',index = False)

In [269]:
from google.colab import files

result_csv.to_csv('Submission.csv')
files.download('Submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>